# Agent Gateway Quickstart

Agent Gateway is a multi-agent framework that offers native support for Snowflake tools. 

The system can be configured to work with 3 types of tools:
- Cortex Search Tool: For unstructured data analysis, which requires a standard RAG access pattern.
- Cortex Analyst Tool: For supporting structured data analysis, which requires a Text2SQL access pattern.
- Python Tool: For supporting custom user operations (using 3rd Party API's), which requires calling arbitrary python.

This notebook walks through how to configure and run a system with all 3 types of tools. The demo is designed to illustrate how the agent can answer questions that require a divserse combination of tools (RAG,Text2SQL, Python, or a combination).

Note that Agent Gateway does not configure the underlying Cortex Search or Cortex Analyst services for the user. Those services must be configured before initializing the agent.

# Agent Configuration

## Connection Setup

Authenticate with Snowpark + set token as environment variable for use by the agents.

In [ ]:
from agent_gateway import Agent
from agent_gateway.tools.logger import gateway_logger
from agent_gateway.tools import CortexAnalystTool, PythonTool, CortexSearchTool
from snowflake.snowpark import Session
import os
from dotenv import load_dotenv

load_dotenv()

print("Loading environment variables...")
if not os.getenv("PRIVATE_KEY_PASSPHRASE"):
    raise ValueError("PRIVATE_KEY_PASSPHRASE environment variable is not set.")


if not os.getenv("SNOWFLAKE_DEFAULT_CONNECTION_NAME"):
    raise ValueError(
        "SNOWFLAKE_DEFAULT_CONNECTION_NAME environment variable is not set."
    )

connection_parameters = {
    "connection_name": os.getenv("SNOWFLAKE_DEFAULT_CONNECTION_NAME"),
    "private_key_file_pwd": os.getenv("PRIVATE_KEY_PASSPHRASE"),
    "database": os.getenv("SNOWFLAKE_DATABASE", "kamesh_llm_demo"),
    "user": os.getenv("SNOWFLAKE_USER", "kameshs"),
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "schema": "DATA",
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH"),
}
snowpark_session = Session.builder.configs(connection_parameters).create()

In [ ]:
snowpark_session.use_database(connection_parameters["database"])
snowpark_session.use_schema(connection_parameters["schema"])
snowpark_session.use_warehouse(connection_parameters["warehouse"])
snowpark_session.sql("ls @my_models").show()

In [ ]:
from sarvamai import SarvamAI
import os
import getpass

SARVAM_AI_TRANSLATE_MODEL = "sarvam-translate:v1"

# Initialize SarvamAI with your API key
if os.getenv("SARVAM_API_KEY") is None:
    sarvam_api_key = getpass.getpass("Enter your SarvamAI API key: ")
else:
    sarvam_api_key = os.getenv("SARVAM_API_KEY")

__client = SarvamAI(api_subscription_key=sarvam_api_key)


def chunk_text(text, max_length=1000):
    """Splits text into chunks of at most max_length characters while preserving word boundaries."""
    chunks = []

    while len(text) > max_length:
        split_index = text.rfind(" ", 0, max_length)  # Find the last space within limit
        if split_index == -1:
            split_index = max_length  # No space found, force split at max_length

        chunks.append(text[:split_index].strip())  # Trim spaces before adding
        text = text[split_index:].lstrip()  # Remove leading spaces for the next chunk

    if text:
        chunks.append(text.strip())  # Add the last chunk

    return chunks


def lang_detect(question: str) -> str | None:
    """
    Detect the language of the given question text using SarvamAI's language identification service.

    This function uses the SarvamAI client to identify the language of the input text and logs
    the detected language code for debugging purposes.

    Args:
        question (str): The input text/question for which to detect the language.

    Returns:
        str | None: The detected language code (e.g., 'en', 'hi', 'ta', etc.) or None if detection fails.

    Example:
        >>> # Detect language of English text
        >>> lang_code = lang_detect("What is the weather today?")
        >>> print(lang_code)
        'en-IN'

        >>> # Detect language of Tamil text
        >>> lang_code = lang_detect("இன்று வானிலை எப்படி இருக்கிறது?")
        >>> print(lang_code)
        'ta-IN'

        >>> # Detect language of Hindi text
        >>> lang_code = lang_detect("आज मौसम कैसा है?")
        >>> print(lang_code)
        'hi-IN'
    """
    response = __client.text.identify_language(input=question)
    __lang_code = response.language_code
    gateway_logger.log("DEBUG", f"Detected Language Code: {__lang_code}\n")

    return __lang_code


def translate(
    lang_code: str,
    question: str,
) -> str:
    """
    Translate text from a detected language to English using SarvamAI's translation service.

    This function takes a language code and question text, logs the translation process,
    and returns the English translation using the SarvamAI translation API.

    Args:
        lang_code (str): The source language code (e.g., 'hi-IN', 'ta-IN', 'te-IN').
        question (str): The input text/question to be translated to English.

    Returns:
        str: The translated text in English.

    Example:
        >>> # Translate Hindi text to English
        >>> hindi_text = "आज मौसम कैसा है?"
        >>> english_translation = translate("hi-IN", hindi_text)
        >>> print(english_translation)
        'How is the weather today?'

        >>> # Translate Tamil text to English
        >>> tamil_text = "இன்று வானிலை எப்படி இருக்கிறது?"
        >>> english_translation = translate("ta-IN", tamil_text)
        >>> print(english_translation)
        'How is the weather today?'

        >>> # Translate Telugu text to English
        >>> telugu_text = "ఈరోజు వాతావరణం ఎలా ఉంది?"
        >>> english_translation = translate("te-IN", telugu_text)
        >>> print(english_translation)
        'How is the weather today?'
    """
    gateway_logger.log(
        "DEBUG",
        f"""\n
Identified Language Code: {lang_code} \n 
Question: {question}\n""",
    )

    response = __client.text.translate(
        input=question,
        source_language_code=lang_code,
        target_language_code="en-IN",
        model=SARVAM_AI_TRANSLATE_MODEL,
    )
    translation = response.translated_text
    gateway_logger.log("DEBUG", f"Translation:{translation}\n")
    return translation


def answer_translator(
    answer: str,
    lang_code: str,
) -> str:
    """
    Translate an English answer back to the original language using SarvamAI's translation service.

    This function takes an English answer and translates it back to the specified target language,
    logging the process for debugging purposes. It's typically used to translate responses back
    to the user's original language after processing.

    Args:
        lang_code (str): The target language code to translate to (e.g., 'hi-IN', 'ta-IN', 'te-IN').
        answer (str): The English text/answer to be translated to the target language.

    Returns:
        str: The translated text in the target language.

    Example:
        >>> # Translate English answer to Hindi
        >>> english_answer = "The weather today is sunny and warm."
        >>> hindi_translation = answer_translator("hi-IN", english_answer)
        >>> print(hindi_translation)
        'आज का मौसम धूप और गर्म है।'

        >>> # Translate English answer to Tamil
        >>> english_answer = "Your support ticket has been resolved."
        >>> tamil_translation = answer_translator("ta-IN", english_answer)
        >>> print(tamil_translation)
        'உங்கள் ஆதரவு டிக்கெட் தீர்க்கப்பட்டது।'

        >>> # Translate English answer to Telugu
        >>> english_answer = "Thank you for your inquiry."
        >>> telugu_translation = answer_translator("te-IN", english_answer)
        >>> print(telugu_translation)
        'మీ విచారణకు ధన్యవాదాలు।'
    """
    gateway_logger.log("DEBUG", f"English answer: \n{answer}\n")
    response = __client.text.translate(
        input=answer,
        source_language_code="en-IN",
        target_language_code=lang_code,
        mode="modern-colloquial",
        model="mayura:v1",
    )
    translation = response.translated_text
    gateway_logger.log("DEBUG", f"Translation:{translation}\n")
    return translation

## Snowflake Tool Configuration

The Cortex Search Tool and the Cortex Analyst Tool need to be configured as follows. Note that a connection object is required for each config. In the case below we're using the same connection object for both because the services are both in the same account/database/schema. Users have the option to pass in different connection objects as needed.

In [ ]:
analyst_config = {
    "semantic_model": "support_tickets.yaml",
    "stage": "MY_MODELS",
    "service_topic": "Customer support tickets model",
    "data_description": "a table with customer support tickets",
    "snowflake_connection": snowpark_session,
    "max_results": 5,
}

search_config = {
    "service_name": "INVOICE_SEARCH",
    "service_topic": "Customer invoice related queries.",
    "data_description": "Customer invoices and related documents",
    "retrieval_columns": ["PARSED_TEXT", "URL"],
    "snowflake_connection": snowpark_session,
    "k": 10,
}

## Python Tool Configuration

Configuring a Python Tool for the Agent Gateway requires 1) Python Callable 2) Tool Description (what does the tool do) 3) Output Description (what does the tool output). 

In the example below we create a NewsTool object that submits a HTTP request to a 3rd Party News API. The python callable is passed into the Python Tool as `news_api_func`.To use the tool below get your free token by signing up for an account at thenewsapi.com or just create your own python function and pass it into the PythonTool.

In [ ]:
__language_identifier_config = {
    "tool_description": "Identify the language of the question",
    "output_description": "It should identify the language code and return it for other tools to use.",
    "python_func": lang_detect,
}

__translator_config = {
    "tool_description": "Use the indentified language from the right tool (e.g., ta-IN for Tamil, hi-IN for Hindi, te-IN for Telugu) as the language code to translate the question to English and then pass the english question to Analyst tool.",
    "output_description": "Returns English translation of the question",
    "python_func": translate,
}

__answer_translator_config = {
    "tool_description": "Translate the answer from English back to the original language of the question.",
    "output_description": "Returns the answer translated from English to the original question's language.",
    "python_func": answer_translator,
}

# Agent Config

In [ ]:
invoice_details = CortexSearchTool(**search_config)
support_tickets = CortexAnalystTool(**analyst_config)
lang_identifier = PythonTool(**__language_identifier_config)
translator = PythonTool(**__translator_config)
answer_translator = PythonTool(**__answer_translator_config)
# margin_eval = SQLTool(**sql_tool_config)


snowflake_tools = [lang_identifier, translator, support_tickets, answer_translator]
# snowflake_tools = [translator,support_tickets,invoice_details,answer_translator]
# agent = Agent(snowflake_connection=snowpark, tools=snowflake_tools, max_retries=3)

# Agent Testing

The 3 types of questions below are designed to showcase the breadth of tool use patterns possible with the Agent Gateway. 

- The Structured Data Questions use the Cortex Analyst agent. 
- The Unstructured Data Questions use either the Cortex Search agent or the Python (News API) agent.
- The last section includes a question that requires the use of both types of tools.

install framework with requisite dependencies with `pip install orchestration-framework[trulens]` and initialize as follows:

In [ ]:
# from agent_gateway import TruAgent
# from trulens.connectors.snowflake import SnowflakeConnector
# from trulens.core.database.connector.default import DefaultDBConnector

# connector = DefaultDBConnector()

# connector = SnowflakeConnector(
#     snowpark_session=snowpark_session,
# )

# agent = TruAgent(
#     app_name="observable",
#     app_version="v0",
#     trulens_snowflake_connection=connector,
#     snowflake_connection=snowpark_session,
#     tools=snowflake_tools,
#     max_retries=3,
# )
FUSION_FINISH = "Finish"
FUSION_REPLAN = "Replan"
# OUTPUT_PROMPT = """
#     "You must solve the Question. You are given Observations and you can use them to solve the Question. "
#     "Then you MUST provide a Thought, and then an Action. Do not use any parenthesis.\n"
#     "You will be given a question either some passages or numbers, which are observations.\n\n"
#     "Thought step can reason about the observations in 1-2 sentences, and Action can be only one type:\n"
#     f" (1) {FUSION_FINISH}(answer): returns the answer and finishes the task using information you found from observations."
#     f" (2) {FUSION_REPLAN}: returns the original user's question, clarifying questions or comments about why it wasn't answered, and replans in order to get the information needed to answer the question."
#     "\n"
#     "Follow the guidelines that you will die if you don't follow:\n"
#     "  - Answer should be directly answer the question.\n"
#     "  - Thought should be 1-2 sentences.\n"
#     "  - Action can only be Finish or Replan\n"
#     "  - Action should be Finish if you have enough information to answer the question\n"
#     "  - Action Should be Replan if you don't have enough information to answer the question\n"
#     "  - You must say <END_OF_RESPONSE> at the end of your response.\n"
#     "  - If the user's question is too vague or unclear, say why and ask for clarification.\n"
#     "  - If the correct tool is used, but the information does not exist, then let the user know.\n"
#     "  - Ensure that langauge of the answer matches language of the user question.\n"
#     "\n"
#     "\n"
#     "Here are some examples:\n"
#     "\n"
#     "Question: What is the EBITDA of Berkshire Hathaway?\n"
#     "cortexanalyst('What is the EBITDA of Berkshire Hathaway?')\n"
#     "Observation:   SYMBOL                    SHORTNAME  START_DATE  END_DATE        EBITDA\n0  BRK-B  Berkshire Hathaway Inc. New      413.72    413.72  107046002688\n"
#     "Thought: Berkshire Hathaway's latest EBITDA is $107,046,002,688, or $107 Billion.\n"
#     f"Action: {FUSION_FINISH}(Berkshire's latest EBITDA is $107 Billion.)\n"
#     "<END_OF_RESPONSE>\n"
#     "\n"
#     "\n"
#     "Question: What is the stock price of the neighborhood laundromat?\n"
#     "cortexanalyst('What is the EBITDA of the neighborhood laundromat?')\n"
#     "Observation: Apologies, but the question 'What is the market cap of the neighborhood laundromat?' is unclear because the company neighborhood laundromat is not specified in the provided semantic data model. The model does not include a company with the name neighborhood laundromate, and without additional information, it is not possible to determine which company the user is referring to.\n"
#     "Thought: The information requested does not exist in the available tools.\n"
#     f"Action: {FUSION_REPLAN}(No data is available for the neighborhood laundromat. Please consider rephrasing your request to be more specific, or contact your administrator to confirm that the system contains the relevant information.)\n"
#     "<END_OF_RESPONSE>\n"
#     "\n"
#     "\n"
#     "Question: What is the latest news about Berkshire Hathaway?\n"
#     "newstool(Berkshire Hathaway)\n"
#     "Observation: '[{'uuid': 'c177ede5-07a7-4f63-a3b7-52790c8fd08e', 'title': 'Berkshire Hathaway-berkshire Hathaway Inc -- Berkshire Says It H…', 'description': 'BERKSHIRE HATHAWAY-BERKSHIRE HATHAWAY INC -- BERKSHIRE SAYS IT HAD $147.4 BLN OF CASH AND EQUIVALENTS AS OF JUNE 30...', 'keywords': 'Markets', 'snippet': \"Berkshire Hathaway Inc. (Berkshire) is a holding company owning subsidiaries engaged in various business activities. Berkshire's various business activities inc...",
#     "url': 'https://www.marketscreener.com/quote/stock/BERKSHIRE-HATHAWAY-INC-11915/news/BERKSHIRE-HATHAWAY-BERKSHIRE-HATHAWAY-INC-BERKSHIRE-SAYS-IT-H-8230-44531571/', 'image_url': 'https://www.marketscreener.com/images/twitter_MS_fdblanc.png', 'language': 'en', 'published_at': '2023-08-05T12:15:15.000000Z', 'source': 'marketscreener.com', 'categories': ['business'], 'relevance_score': 55.634586}, {'uuid': '56202cf0-38af-4a20-b411-994c92d8c7cd', 'title': 'Berkshire Hathaway Inc. (OTCMKTS:BRK-A) Major Shareholder Berkshire Hathaway Inc Acquires 716,355 Shares', 'description': 'Read Berkshire Hathaway Inc. (OTCMKTS:BRK-A) Major Shareholder Berkshire Hathaway Inc Acquires 716,355 Shares at ETF Daily News', 'keywords': 'Berkshire Hathaway, OTCMKTS:BRK-A, BRK-A, Financial Service, Insider Trading, Insider Trades, Stocks', 'snippet': 'Berkshire Hathaway Inc. (OTCMKTS:BRK-A – Get Rating) major shareholder Berkshire Hathaway Inc bought 716,355 shares of Berkshire Hathaway stock in a transacti...', 'url': 'https://www.etfdailynews.com/2022/05/13/berkshire-hathaway-inc-otcmktsbrk-a-major-shareholder-berkshire-hathaway-inc-acquires-716355-shares/', 'image_url': 'https://www.americanbankingnews.com/wp-content/timthumb/timthumb.php?src=https://www.marketbeat.com/logos/berkshire-hathaway-inc-logo.png?v=20211203153558&w=240&h=240&zc=2', 'language': 'en', 'published_at': '2022-05-13T11:18:50.000000Z', 'source': 'etfdailynews.com', 'categories': ['business'], 'relevance_score': 53.612434}]"
#     "\n"
#     "Thought: The recent news about Berkshire Hathaway include information about its financials and recent activities.\n"
#     f"Action: {FUSION_FINISH}('Recent news about Berkshire Hathaways includes:\n- Article: Berkshire Hathaway-Berkshire Hathaway Inc -- Berkshire Says It H…  Source: [Market Screener](https://www.marketscreener.com/quote/stock/BERKSHIRE-HATHAWAY-INC-11915/news/BERKSHIRE-HATHAWAY-BERKSHIRE-HATHAWAY-INC-BERKSHIRE-SAYS-IT-H-8230-44531571/) \n - Article: Berkshire Hathaway Inc. (OTCMKTS:BRK-A) Major Shareholder Berkshire Hathaway Inc Acquires 716,355 Shares' Source: [ETF Daily News](https://www.etfdailynews.com/2022/05/13/berkshire-hathaway-inc-otcmktsbrk-a-major-shareholder-berkshire-hathaway-inc-acquires-716355-shares/)) '\n"
#     "<END_OF_RESPONSE>\n"
#     "\n"
#     "\n"
#     "Question: How many queries are processed on Snowflake's platform?\n"
#     "cortexsearch(How many queries are processed on Snowflake's platform?)\n"
#     "Observation: ['deliver the Data Cloud, enabling a consistent, global user experience.\nOur platform supports a wide range of workloads that enable our customers’ most important business objectives, including data warehousing, data lakes, data engineering, data\nscience, data application development, and data sharing. From January 1, 2022 to January 31, 2022, we processed an average of over 1,496 million daily queries across all of our\ncustomer accounts, up from an average of over 777 million daily queries during the corresponding month of the prior fiscal year. We also recently launched our Powered by\nSnowflake program to help companies build, operate, and grow applications in the Data Cloud by supporting developers across all stages of the application journey. Members of the\nprogram have access to go-to-market, customer support, and engineering expertise.\nWe have an industry-vertical focus, which allows us to go to market with tailored business solutions. For example, we have launched the Financial Services Data Cloud, the\nMedia Data Cloud, the Healthcare and Life Sciences Data Cloud, and the Retail Data Cloud. Each of these Data Clouds brings together Snowflake’s platform capabilities with\nindustry-specific partner solutions and datasets to drive business growth and deliver improved experiences and insights.\nOur business benefits from powerful network effects. The Data Cloud will continue to grow as organizations move their siloed data from cloud-based repositories and on-\npremises data centers to the Data Cloud. The more customers adopt our platform, the more data can be exchanged with other Snowflake customers, partners, data providers, and\ndata consumers, enhancing the value of our platform for all users. We believe this network effect will help us drive our vision of the Data Cloud.\n75/14/24, 8:55 AM snow-20220131\nhttps://www.sec.gov/Archives/edgar/data/1640147/000164014722000023/snow-20220131.htm 8/183Table of Contents',"
#     "'the Data Cloud, enabling a consistent, global user experience.\nOur platform supports a wide range of workloads that enable our customers’ most important business objectives, including data warehouse, data lake, data engineering, AI/ML,\napplications, collaboration, cybersecurity and Unistore. From January 1, 2024 to January 31, 2024, we processed an average of approximately 4.2 billion daily queries across all our\ncustomer accounts, up from an average of approximately 2.6 billion daily queries during the corresponding month of the prior fiscal year. We are committed to expanding our\nplatform’s use cases and supporting developers in building their applications and businesses. In 2021, we launched Snowpark for Java and Scala to allow developers to build in the\nlanguage of their choice, and in 2022 we added support for Python. In 2023, we launched Snowpark Container Services, a fully managed container platform designed to facilitate\nthe deployment, management, and scaling of containerized applications and AI/ML models within our ecosystem. We continue to invest in our Native Application program to help\ncompanies build, operate, and market applications in the Data Cloud by supporting developers across all stages of the application journey.\nWe have an industry-vertical focus, which allows us to go to market with tailored business solutions. For example, we have launched the Telecom Data Cloud, the Financial\nServices Data Cloud, the Media Data Cloud, the Healthcare and Life Sciences Data Cloud, and the Retail Data Cloud. Each of these brings together Snowflake’s platform\ncapabilities with industry-specific partner solutions and datasets to drive business growth and deliver improved experiences and insights.\nOur business benefits from powerful network effects. The Data Cloud will continue to grow as organizations move their siloed data from cloud-based repositories and on-',"
#     "'Our cloud-native architecture consists of three independently scalable but logically integrated layers across compute, storage, and cloud services. The compute layer provides\ndedicated resources to enable users to simultaneously access common data sets for many use cases with minimal latency. The storage layer ingests massive amounts and varieties of\nstructured, semi-structured, and unstructured data to create a unified data record. The cloud services layer intelligently optimizes each use case’s performance requirements with no\nadministration. This architecture is built on three major public clouds across 38 regional deployments around the world. These deployments are generally interconnected to deliver\nthe Data Cloud, enabling a consistent, global user experience.\nOur platform supports a wide range of workloads that enable our customers’ most important business objectives, including data warehousing, data lakes, and Unistore, as well\nas collaboration, data engineering, cybersecurity, data science and machine learning, and application development. From January 1, 2023 to January 31, 2023, we processed an\naverage of approximately 2.6 billion daily queries across all our customer accounts, up from an average of approximately 1.5 billion daily queries during the corresponding month\nof the prior fiscal year. We are committed to expanding our platform’s use cases and supporting developers in building their applications and businesses. In 2021, we launched\nSnowpark for Java to allow developers to build in the language of their choice, and in 2022 we added support for Python. We continue to invest in our Powered by Snowflake\nprogram to help companies build, operate, and market applications in the Data Cloud by supporting developers across all stages of the application journey. As of January 31, 2023,\nwe had over 820 Powered by Snowflake registrants. Powered by Snowflake partners have access to go-to-market, customer support, and engineering expertise.',"
#     "'performance comparable to a relational, structured representation.\n•Query Capabilities. Our platform is engineered to query petabytes of data. It implements support for a large subset of the ANSI SQL standard for read operations and data\nmodification operations. Our platform provides additional features, including:\n◦Time travel. Our platform keeps track of all changes happening to a table, which enables customers to query previous versions based on their preferences. Customers\ncan query as of a relative point in time or as of an absolute point in time. This has a broad array of use cases for customers, including error recovery, time-based\nanalysis, and data quality checks.5/14/24, 8:59 AM snow-20210131\nhttps://www.sec.gov/Archives/edgar/data/1640147/000164014721000073/snow-20210131.htm 17/193◦ Cloning. Our architecture enables us to offer zero-copy cloning, an operation by which entire tables, schemas, or databases can be duplicated—or cloned—without\nhaving to copy or duplicate the underlying data. Our platform leverages the separation between cloud services and storage to be able to track independent clones of\nobjects sharing the same physical copy of the underlying data. This enables a variety of customer use cases such as making copies of production data for data\nscientists, creating custom snapshots in time, or testing data pipelines.\n105/14/24, 8:59 AM snow-20210131\nhttps://www.sec.gov/Archives/edgar/data/1640147/000164014721000073/snow-20210131.htm 18/193Table of Contents\n•Compute Model. Our platform offers a variety of capabilities to operate on data, from ingestion to transformation, as well as rich query and analysis. Our compute services\nare primarily presented to users in one of two models, either through explicit specification of compute clusters we call virtual warehouses or through a number of serverless\nservices.',"
#     "'performance.\n◦ Metadata. When data is ingested, our platform automatically extracts and stores metadata to speed up query processing. It does so by collecting data distribution\ninformation for all columns in every micro-partition.5/14/24, 8:55 AM snow-20240131\nhttps://www.sec.gov/Archives/edgar/data/1640147/000164014724000101/snow-20240131.htm 20/217◦Semi-structured and unstructured data. In addition to structured, relational data, our platform supports semi-structured data, including JSON, Avro, and Parquet, and\nunstructured data, including PDF documents, screenshots, recordings, and images. Data in these formats can be ingested and queried with performance comparable to a\nrelational, structured representation.\n135/14/24, 8:55 AM snow-20240131\nhttps://www.sec.gov/Archives/edgar/data/1640147/000164014724000101/snow-20240131.htm 21/217Table of Contents\n•Query Capabilities. Our platform is engineered to query petabytes of data. It implements support for a large subset of the ANSI SQL standard for read operations and data\nmodification operations. Our platform provides additional features, including:\n◦Time travel. Our platform keeps track of all changes happening to a table, which enables customers to query previous versions based on their preferences. Customers\ncan query as of a relative point in time or as of an absolute point in time. This has a broad array of use cases for customers, including error recovery, time-based\nanalysis, and data quality checks.\n◦ Cloning. Our architecture enables us to offer zero-copy cloning, an operation by which entire tables, schemas, or databases can be duplicated—or cloned—without\nhaving to copy or duplicate the underlying data. Our platform leverages the separation between cloud services and storage to be able to track independent clones of\nobjects sharing the same physical copy of the underlying data. This enables a variety of customer use cases such as making copies of production data for data']"
#     "Thought: Berkshire Hathaway's latest EBITDA is $107,046,002,688, or $107 Billion.\n"
#     f"Action: {FUSION_FINISH}(Based on January 2024 data, Snowflake processes an average of approximately 4.2 billion daily queries across all customer account. This is an increase up from an average of approximately 2.6 billion daily queries during the corresponding month of the prior year.)\n"
#     "<END_OF_RESPONSE>\n"
#     "\n"
#     "\n",
# """

OUTPUT_PROMPT = """
You are a helpful multi lingual analyst who can answer about customer support tickets. Guidelines for your tasks will be:
1. Translates the question from detected language to english
2. Send the translated question to the CortexAnalystTool to get the answer.
3. Translate the answer back to the original language of the question using the 'answer_translator' tool.
4. The answer should start with a marker 'Action: Finish' and end with a marker '<END_OF_RESPONSE>' with actual answer between the start and end marker.
"""

agent = Agent(
    snowflake_connection=snowpark_session,
    agent_llm="claude-3-5-sonnet",
    tools=snowflake_tools,
    fusion_prompt=OUTPUT_PROMPT,
)

## Structured Data Questions

In [ ]:
out = agent(
    "செல்லுலார் சேவை வகைக்கும் வணிக இணைய சேவை வகைக்கும் இடையிலான வாடிக்கையாளர் ஆதரவு டிக்கெட்டுகளின் விநியோகத்தை எனக்குக் காட்ட முடியுமா?"
)
gateway_logger.log("DEBUG", f"Output: {out}\n")

# Unstructured Data Questions

In [ ]:
out = agent("கிரிகோரி ரோமிங் கட்டணம் வசூலிக்கப்பட்டாரா?")
gateway_logger.log("INFO", "Agent Output:")
gateway_logger.log("INFO", out["output"])

In [ ]:
out = agent("గ్రెగొరీకి ఛార్జ్ చేసిన రోమింగ్ ఫీజు ఉందా, ఉంటే ఆ మొత్తం ఎంత మరియు ఏ ఇన్వాయిస్ అని చెప్పగలరా?")
gateway_logger.log("INFO", "Agent Output:")
gateway_logger.log("INFO", out["output"])

# Run the dashboard to view traces

In [ ]:
# from trulens.core import TruSession
# from trulens.dashboard import run_dashboard

# session = TruSession(connector=connector)

# run_dashboard(session, port=8084, force=True)